In [1]:
import pandas as pd
import numpy as np

In [2]:
#execute nlp2 before this to get new sets
x_train = pd.read_pickle('./train-both.pkl')
x_test = pd.read_pickle('./test-both.pkl')
y_train = pd.read_pickle('./y_train-both.pkl')
y_test = pd.read_pickle('./y_test-both.pkl')
test_pred = pd.read_pickle('./pred-test-both-lgbm.pkl')
train_pred = pd.read_pickle('./pred-train-both-lgbm.pkl')

In [16]:
print(test_pred.shape,train_pred.shape)

(6829, 20) (38697, 20)


In [3]:
y_predtest_new = np.subtract(y_test,test_pred)
y_predtrain_new = np.subtract(y_train,train_pred)

In [4]:
from gensim.models import Word2Vec
model = Word2Vec.load('model1500-10-3.bin')

In [5]:
WordVectorz=dict(zip(model.wv.index2word,model.wv.vectors))
class AverageEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim =100 # because we use 100 embedding points 

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])


In [6]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/akshara/anaconda3/envs/mimic/lib/python3

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6707589282982757554
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 9206601737086141156
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 46071808
locality {
  bus_id: 1
  links {
  }
}
incarnation: 3304812428548568954
physical_device_desc: "device: 0, name: Tesla M40 24GB, pci bus id: 0000:04:00.0, compute capability: 5.2"
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 610205696
locality {
  bus_id: 2
  numa_node: 1
  links {
  }
}
incarnation: 5980804884914399148
physical_device_desc: "device: 1, name: Tesla M40 24GB, pci bus id: 0000:82:00.0, compute capability: 5.2"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 9638976143799774001
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_GPU:1"
device_type: "

In [7]:
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier,XGBRegressor
from lightgbm import LGBMClassifier,LGBMRegressor
from catboost import CatBoostClassifier,CatBoostRegressor
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import ClassifierChain
from sklearn.model_selection import GridSearchCV,StratifiedKFold
from sklearn.metrics import jaccard_score,roc_auc_score,confusion_matrix,hamming_loss
from sklearn.multioutput import MultiOutputRegressor
#model1 = CatBoostClassifier(iterations=1000, task_type="GPU",devices='0:1',learning_rate=0.1)
#d_train = lgb.Dataset(x_train, label=y_train)
#clf_multilabel = OneVsRestClassifier(LGBMClassifier(iterations=100))
#pipe1=Pipeline([("wordVectz",AverageEmbeddingVectorizer(WordVectorz)),("multilabel",MultiOutputRegressor(
#    XGBRegressor(iterations=1000,learning_rate=0.15,n_jobs=-1)))])

clf_multiregressor = MultiOutputRegressor(LGBMRegressor(iterations=1000,learning_rate=0.1))
#clf_multiregressor.fit(x_train,y_predtrain_new)

In [8]:
#x_train.shape

In [9]:
clf_multiregressor.fit(x_train,y_predtrain_new)

MultiOutputRegressor(estimator=LGBMRegressor(boosting_type='gbdt',
                                             class_weight=None,
                                             colsample_bytree=1.0,
                                             importance_type='split',
                                             iterations=1000, learning_rate=0.1,
                                             max_depth=-1, min_child_samples=20,
                                             min_child_weight=0.001,
                                             min_split_gain=0.0,
                                             n_estimators=100, n_jobs=-1,
                                             num_leaves=31, objective=None,
                                             random_state=None, reg_alpha=0.0,
                                             reg_lambda=0.0, silent=True,
                                             subsample=1.0,
                                             subsample_for_bin=200000,
  

In [10]:
y_pred_new = clf_multiregressor.predict(x_test)

In [11]:
y_combine = np.add(y_pred_new,y_predtest_new)

In [22]:
y_combine  = pd.DataFrame(y_combine)
y_combine.to_pickle('./y_combine_nlp_pred-both-2_lgbm-lgbm.pkl')

In [17]:
y_combine.shape

(6829, 20)

In [12]:
y_combine_pred = y_combine.round()  


In [13]:
np.unique(y_combine_pred)

array([-2., -1.,  0.,  1.,  2.])

In [14]:
y_combine_pred = np.where(y_combine_pred<=0,0,1)

In [15]:
from sklearn.metrics import jaccard_score,roc_auc_score,confusion_matrix,hamming_loss
#cm = confusion_matrix(y_test,y_pred) 
#Classification metrics can't handle a mix of multilabel-indicator and unknown targets
print(jaccard_score(y_test,y_combine_pred,average='micro'))
print(jaccard_score(y_test,y_combine_pred,average='macro'))
print(roc_auc_score(y_test,y_combine_pred))
print(hamming_loss(y_test,y_combine_pred))


0.6755715203273307
0.8058115600785116
0.9058644183727136
0.09985356567579441
